In [36]:
import numpy as np 
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
from sklearn.model_selection import train_test_split,GridSearchCV
import pickle
from sklearn.ensemble import RandomForestRegressor
from sklearn.metrics import r2_score


In [37]:
#loading the dataset
df=pd.read_csv('Car_Price_Prediction.csv')
df

,Make,Model,Year,Engine Size,Mileage,Fuel Type,Transmission,Price
0,Honda,Model B,2015,3.9,74176,Petrol,Manual,30246.207931
1,Ford,Model C,2014,1.7,94799,Electric,Automatic,22785.747684
2,BMW,Model B,2006,4.1,98385,Electric,Manual,25760.290347
3,Honda,Model B,2015,2.6,88919,Electric,Automatic,25638.003491
4,Honda,Model C,2004,3.4,138482,Petrol,Automatic,21021.386657
...,...,...,...,...,...,...,...,...
995,Toyota,Model D,2002,1.9,5445,Petrol,Manual,22765.597091
996,Honda,Model B,2020,3.1,149112,Diesel,Manual,30392.575567
997,Ford,Model C,2008,1.9,195387,Petrol,Automatic,16446.892292
998,Toyota,Model A,2003,4.4,246,Petrol,Automatic,27396.156708


In [38]:
df.info 


<bound method DataFrame.info of        Make    Model  Year  Engine Size  Mileage Fuel Type Transmission  \
0     Honda  Model B  2015          3.9    74176    Petrol       Manual   
1      Ford  Model C  2014          1.7    94799  Electric    Automatic   
2       BMW  Model B  2006          4.1    98385  Electric       Manual   
3     Honda  Model B  2015          2.6    88919  Electric    Automatic   
4     Honda  Model C  2004          3.4   138482    Petrol    Automatic   
..      ...      ...   ...          ...      ...       ...          ...   
995  Toyota  Model D  2002          1.9     5445    Petrol       Manual   
996   Honda  Model B  2020          3.1   149112    Diesel       Manual   
997    Ford  Model C  2008          1.9   195387    Petrol    Automatic   
998  Toyota  Model A  2003          4.4      246    Petrol    Automatic   
999    Audi  Model C  2020          2.1    31153    Diesel    Automatic   

            Price  
0    30246.207931  
1    22785.747684  
2    25

In [39]:
df=df.drop(['Model'],axis=1)
df.head()

,Make,Year,Engine Size,Mileage,Fuel Type,Transmission,Price
0,Honda,2015,3.9,74176,Petrol,Manual,30246.207931
1,Ford,2014,1.7,94799,Electric,Automatic,22785.747684
2,BMW,2006,4.1,98385,Electric,Manual,25760.290347
3,Honda,2015,2.6,88919,Electric,Automatic,25638.003491
4,Honda,2004,3.4,138482,Petrol,Automatic,21021.386657


In [40]:
df['Car_Age'] = 2025 - df['Year']
df.drop('Year', axis=1, inplace=True)
df.head()


,Make,Engine Size,Mileage,Fuel Type,Transmission,Price,Car_Age
0,Honda,3.9,74176,Petrol,Manual,30246.207931,10
1,Ford,1.7,94799,Electric,Automatic,22785.747684,11
2,BMW,4.1,98385,Electric,Manual,25760.290347,19
3,Honda,2.6,88919,Electric,Automatic,25638.003491,10
4,Honda,3.4,138482,Petrol,Automatic,21021.386657,21


In [41]:
df = df.rename(columns={'Make': 'Model'}) # rename make wit model
df


,Model,Engine Size,Mileage,Fuel Type,Transmission,Price,Car_Age
0,Honda,3.9,74176,Petrol,Manual,30246.207931,10
1,Ford,1.7,94799,Electric,Automatic,22785.747684,11
2,BMW,4.1,98385,Electric,Manual,25760.290347,19
3,Honda,2.6,88919,Electric,Automatic,25638.003491,10
4,Honda,3.4,138482,Petrol,Automatic,21021.386657,21
...,...,...,...,...,...,...,...
995,Toyota,1.9,5445,Petrol,Manual,22765.597091,23
996,Honda,3.1,149112,Diesel,Manual,30392.575567,5
997,Ford,1.9,195387,Petrol,Automatic,16446.892292,17
998,Toyota,4.4,246,Petrol,Automatic,27396.156708,22


,Engine Size,Mileage,Price,Car_Age
count,1000.000000,1000.00000,1000.000000,1000.000000
mean,2.798300,97192.48700,25136.615530,14.312000
std,1.024137,59447.31576,5181.401368,6.288577
min,1.000000,56.00000,6704.953524,4.000000
25%,1.900000,44768.75000,21587.878370,9.000000
50%,2.800000,94411.50000,25189.325247,14.000000
75%,3.700000,148977.75000,28806.368974,20.000000
max,4.500000,199867.00000,41780.504635,25.000000


In [54]:
df.describe()

,Model,Engine Size,Mileage,Fuel Type,Transmission,Price,Car_Age
count,1000.000000,1000.000000,1000.00000,1000.000000,1000.000000,1000.000000,1000.000000
mean,1.970000,2.798300,97192.48700,0.987000,0.511000,25136.615530,14.312000
std,1.404661,1.024137,59447.31576,0.821892,0.500129,5181.401368,6.288577
min,0.000000,1.000000,56.00000,0.000000,0.000000,6704.953524,4.000000
25%,1.000000,1.900000,44768.75000,0.000000,0.000000,21587.878370,9.000000
50%,2.000000,2.800000,94411.50000,1.000000,1.000000,25189.325247,14.000000
75%,3.000000,3.700000,148977.75000,2.000000,1.000000,28806.368974,20.000000
max,4.000000,4.500000,199867.00000,2.000000,1.000000,41780.504635,25.000000


In [43]:
df.isnull().sum()

Model           0
Engine Size     0
Mileage         0
Fuel Type       0
Transmission    0
Price           0
Car_Age         0
dtype: int64

In [44]:
copied_dataset=df.copy()
from sklearn.preprocessing import LabelEncoder
obj=LabelEncoder()
for i in df.columns:
    if df[i].dtype==object:
        df[i]=obj.fit_transform(df[i])
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1000 entries, 0 to 999
Data columns (total 7 columns):
 #   Column        Non-Null Count  Dtype  
---  ------        --------------  -----  
 0   Model         1000 non-null   int64  
 1   Engine Size   1000 non-null   float64
 2   Mileage       1000 non-null   int64  
 3   Fuel Type     1000 non-null   int64  
 4   Transmission  1000 non-null   int64  
 5   Price         1000 non-null   float64
 6   Car_Age       1000 non-null   int64  
dtypes: float64(2), int64(5)
memory usage: 54.8 KB


In [45]:
df.head()

,Model,Engine Size,Mileage,Fuel Type,Transmission,Price,Car_Age
0,3,3.9,74176,2,1,30246.207931,10
1,2,1.7,94799,1,0,22785.747684,11
2,1,4.1,98385,1,1,25760.290347,19
3,3,2.6,88919,1,0,25638.003491,10
4,3,3.4,138482,2,0,21021.386657,21


In [46]:
copied_dataset.to_csv('Copied_Car_Price_Prediction.csv')


In [47]:
copied_dataset

,Model,Engine Size,Mileage,Fuel Type,Transmission,Price,Car_Age
0,Honda,3.9,74176,Petrol,Manual,30246.207931,10
1,Ford,1.7,94799,Electric,Automatic,22785.747684,11
2,BMW,4.1,98385,Electric,Manual,25760.290347,19
3,Honda,2.6,88919,Electric,Automatic,25638.003491,10
4,Honda,3.4,138482,Petrol,Automatic,21021.386657,21
...,...,...,...,...,...,...,...
995,Toyota,1.9,5445,Petrol,Manual,22765.597091,23
996,Honda,3.1,149112,Diesel,Manual,30392.575567,5
997,Ford,1.9,195387,Petrol,Automatic,16446.892292,17
998,Toyota,4.4,246,Petrol,Automatic,27396.156708,22


In [48]:
X=df.drop('Price',axis=1)
y=df['Price']

In [49]:
Xtrain,Xtest,ytrain,ytest=train_test_split(X,y,test_size=0.2,random_state=42) # splitting the dataset into training and testing sets


In [50]:
from sklearn.linear_model import LinearRegression,Ridge,Lasso
model=LinearRegression()
model.fit(Xtrain,ytrain)

LinearRegression()

In [51]:
print('training:',model.score(Xtrain,ytrain))
print('testing:',model.score(Xtest,ytest))

training: 0.8441496973329212
testing: 0.8210721096901781


In [52]:
import pickle
with open('rfmodel.pkl','wb') as file: # saving the trained linear regressor model as a pickle file
    pickle.dump(model,file)
    